In [126]:
# Imports
import numpy as np

# PCA on data for feature selection / Avoid collinearity

In [127]:
# Implementation of PCA using numpy
def PCA_dim_reduction(train_file, test_file, n_components):
    
    # Read data
    train = np.loadtxt(train_file, skiprows=1, delimiter=",")
    test = np.loadtxt(test_file, skiprows=1, delimiter=",")

    train_x = train[:,:-1]  
    train_y = train[:,-1]

    test_x = test[:,:-1]
    test_y = test[:,-1]
    
    # Standardize data
    mean = np.mean(train_x, axis=0)
    std = np.std(train_x, axis=0)
    
    train_x = (train_x - mean) / std
    test_x = (test_x - mean) / std
    
    # Get covariance matrix
    covmat = np.cov(train_x.T)
    
    # Get eigenvalues
    eigval, eigvec = np.linalg.eig(covmat)
    
    # Get explained variance (if needed)
    exp_var = []
    for i in range(len(eigval)):
        exp_var.append(eigval[i]/np.sum(eigval))
    exp_var = sorted(exp_var, reverse=True)

    # Pair up eigenvalues and vectors, so the vectors can be ranked by values
    eigenpairs = []
    for i in range(len(eigval)):
        eigenpairs.append((eigval[i]/np.sum(eigval), eigvec[:, i].T))
    eigenpairs = sorted(eigenpairs, reverse=True)
    
    # Put in the vectors from most to least important in a list
    sorted_vectors = []
    for i in range(len(eigenpairs)):
        sorted_vectors.append(eigenpairs[i][1])
    
    # Make new variables to store the transformed data (the n principle coponents)
    new_train_x = np.zeros((len(train_x), n_components))
    new_test_x = np.zeros((len(test_x), n_components))
    
    # Transform the data into n principle components on the new subspace
    for i in range(n_components):
        new_train_x[:,i] = train_x @ sorted_vectors[i]
        new_test_x[:,i] = test_x @ sorted_vectors[i]
    
    # Store transformed data (pc's) in old variables
    train_x = new_train_x
    test_x = new_test_x
    
    # Return the transformed train and test data + class labels
    return train_x, train_y, test_x, test_y

# QDA Implementation

In [128]:
# Function to put the class specific data into designated arrays
def preprocess_QDA(train_x, train_y):
    # Collecting all class labels
    classes = np.unique(train_y)

    # Seperating all class-specific observations onto designated variables and finally collecting them into an array.
    mask = train_y == 1
    class1 = train_x[mask, :]

    mask = train_y == 2
    class2 = train_x[mask, :]

    mask = train_y  == 3
    class3 = train_x[mask, :]

    mask = train_y == 5
    class5 = train_x[mask, :]

    mask = train_y == 6
    class6 = train_x[mask, :]

    mask = train_y == 7
    class7 = train_x[mask, :]
    
    # list of arrays with observations from specific classes
    class_list = [class1, class2, class3, class5, class6, class7]
    
    # Return list with arrays of observations in classes
    return class_list


# Function to get the class parameters: mean, covariance matrix and probability of class
def get_parameters_QDA(class_list, train_x):
    
    # Estimate a mean vector per class
    class_means = np.zeros((len(class_list), train_x.shape[1]))
    for i in range(len(class_list)):
        class_means[i] = class_list[i].mean(axis=0)

    # Estimate a covariance matrix per class
    class_covmats = []
    for i in range(len(class_list)):
        class_covmats.append(np.cov(class_list[i].T))


    # Estimate class probabilities
    class_probs = []
    for i in range(len(class_list)):
        class_probs.append(len(class_list[i]) / len(train_x))
    
    # Return the parameters for each class in designated lists
    return class_means, class_covmats, class_probs


# Discriminant function from slides
# It takes in a specific observation and the parameters of a specific class
def QDA_discriminant_function(x, class_mean, class_covmat, class_prob):
    
    a_k = 2 * np.log(class_prob) - np.log(np.linalg.det(class_covmat)) - class_mean.T @ np.linalg.inv(class_covmat) @ class_mean
    b_k = 2 * class_mean.T @ np.linalg.inv(class_covmat)
    c_k = -np.linalg.inv(class_covmat)
    
    # Returns discriminant function for a specific class on a specific observation
    return a_k + b_k.T @ x + x.T @ c_k @ x


# QDA scoring by finding the class that maximises the dicriminant function for each observation
def QDA_class_scoring(test_x, test_y, class_list, covmat_list, mean_list, prob_list):
    
    classes = np.unique(test_y)
    class_scores = np.zeros((len(test_x), len(class_list)))
    
    for x in range(len(test_x)):
        for k in range(len(class_list)):
            # formula from lectures
            class_scores[x, k] = QDA_discriminant_function(test_x[x,:], mean_list[k], covmat_list[k], prob_list[k])
            
            # formula from sklearn library
            #class_scores[x, k] = log_posterior_sklearn(test_x[x,:], mean_list[k], covmat_list[k], prob_list[k])
    
    # List of classes that maximises the dicsriminant function for each observation
    class_max = []
    for row in class_scores:
        class_max.append(int(classes[np.argmax(row)]))
    
    # returns list with the class that maximised the discriminant function for each observation
    return class_max

# Functions to get metrics

In [129]:
# Function that computes a confusion matrix which is used to compute the below functions
def cm_maker(y, ypred, n_classes):
    
    low = [1, 2, 3]
    high = [5, 6, 7]
    
    cm = np.zeros((n_classes, n_classes))
    
    for i, j in zip(ypred, y):
        
        if i in low:
            i = i - 1
        if i in high:
            i = i - 2
            
        if j in low:
            j = j - 1
        if j in high:
            j = j - 2
            
        cm[int(i), int(j)] += 1
        
    return cm


# Function computes precision score
def preci(cm, c):
    
    if sum(cm[c,:]) == 0:
        return 0
    else:
        return cm[c,c]/sum(cm[c,:])


# Function computes recall score
def recall(cm, c):
    
    return cm[c,c]/sum(cm[:,c])


# Function computes f1-score
def f1(cm, c):
    if (preci(cm,c) + recall(cm,c)) == 0:
        return 0
    else: 
        return 2 * (preci(cm,c) * recall(cm,c)) / (preci(cm,c) + recall(cm,c))
    

# Function computes weighted f1-score
def weighted_f1(cm, n_classes):
    co_su=cm.sum(axis=0)
    n=cm.sum()
    
    weighted_f1_sum = 0
    
    for c in range(n_classes):
        if co_su[c] != 0:
            weighted_f1_sum += f1(cm, c) * co_su[c] / n

    return round(weighted_f1_sum, 3)
        

# Function computes macro f1-score
def macro_f1(cm, n_classes):
    
    f1_sum = 0
    
    for i in range(n_classes):
        f1_sum += f1(cm, i)
    
    return round(f1_sum / n_classes, 3)


# Function to get accuracy
def accuracy(test_y, ypred):
    
    # Count of times where true labels equal predictions
    true_positives = 0
    for i in range(len(test_y)):
        if test_y[i] == ypred[i]:
            true_positives += 1
    
    return true_positives / len(test_y)


# Combines functions above for a coherent performance report
def performance_report(test_y, ypred, n_classes):
    
    class_labels = [1, 2, 3, 5, 6, 7]
    cm = cm_maker(test_y, ypred, n_classes)
    
    print('\nConfusion matrix for prediction:\n', cm)
    print('\n\nAccuracy for prediction:\n', accuracy(test_y, ypred))
    
    print('\n\nMetrics for classes')
    print('_______________________________________________________________________________')
    print('Class\t|\tPrecision\t|\tRecall\t\t|\tF1 Score')
    print('_______________________________________________________________________________')
    
    for i in range(n_classes):
        
        print('\nClass', class_labels[i],'|\t',round(preci(cm, i), 3),'\t\t|\t',round(recall(cm, i), 3),'\t\t|\t',round(f1(cm, i), 3))
    
    print('\n\nWeighted F1 score:\n', weighted_f1(cm, n_classes))
    print('\nMacro F1 score:\n', macro_f1(cm, n_classes))
    

# QDA with PCA data

In [130]:
# Get PC's from train/test data
train_x, train_y, test_x, test_y = PCA_dim_reduction('df_train.csv', 'df_test.csv', 5)

# Get class observations in arrays in a list
class_list = preprocess_QDA(train_x, train_y)

# Get class means, covariance matrices and probabilities
mean_list, covmat_list, prob_list = get_parameters_QDA(class_list, train_x)

# Get y predictions (class that maximises the discriminant function for each observation)
ypred = QDA_class_scoring(test_x, test_y, class_list, covmat_list, mean_list, prob_list)

# Get performance report on the classification metrics
performance_report(test_y, ypred, len(class_list))


Confusion matrix for prediction:
 [[20. 14.  4.  0.  0.  0.]
 [ 0.  7.  0.  1.  1.  0.]
 [ 1.  1.  1.  0.  0.  0.]
 [ 0.  1.  0.  3.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  2.  8.]]


Accuracy for prediction:
 0.6


Metrics for classes
_______________________________________________________________________________
Class	|	Precision	|	Recall		|	F1 Score
_______________________________________________________________________________

Class 1 |	 0.526 		|	 0.952 		|	 0.678

Class 2 |	 0.778 		|	 0.304 		|	 0.438

Class 3 |	 0.333 		|	 0.2 		|	 0.25

Class 5 |	 0.6 		|	 0.75 		|	 0.667

Class 6 |	 0 		|	 0.0 		|	 0

Class 7 |	 0.8 		|	 0.889 		|	 0.842


Weighted F1 score:
 0.551

Macro F1 score:
 0.479


# QDA with sklearn on PCA data

In [131]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
qda = QuadraticDiscriminantAnalysis(store_covariance=True)

train_x, train_y, test_x, test_y = PCA_dim_reduction('df_train.csv', 'df_test.csv', 5)

qda.fit(train_x, train_y)

ypred = qda.predict(test_x)

performance_report(test_y, ypred, 6)


Confusion matrix for prediction:
 [[20. 14.  4.  0.  0.  0.]
 [ 0.  7.  0.  1.  1.  0.]
 [ 1.  1.  1.  0.  0.  0.]
 [ 0.  1.  0.  3.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  2.  8.]]


Accuracy for prediction:
 0.6


Metrics for classes
_______________________________________________________________________________
Class	|	Precision	|	Recall		|	F1 Score
_______________________________________________________________________________

Class 1 |	 0.526 		|	 0.952 		|	 0.678

Class 2 |	 0.778 		|	 0.304 		|	 0.438

Class 3 |	 0.333 		|	 0.2 		|	 0.25

Class 5 |	 0.6 		|	 0.75 		|	 0.667

Class 6 |	 0 		|	 0.0 		|	 0

Class 7 |	 0.8 		|	 0.889 		|	 0.842


Weighted F1 score:
 0.551

Macro F1 score:
 0.479


# QDA when combining data into window/non-window

In [132]:
# Read in the top 5 Principal Components
train_x, train_y, test_x, test_y = PCA_dim_reduction('df_train.csv', 'df_test.csv', 5)
    
# Divide test and train data into features and class labels
window = [1,2,3]

for i in range(len(train_y)):
    if train_y[i] in window:
        train_y[i] = 1
    else:
        train_y[i] = 2

for i in range(len(test_y)):
    if test_y[i] in window:
        test_y[i] = 1
    else:
        test_y[i] = 2
        
mask = train_y == 1
class1 = train_x[mask, :]

mask = train_y == 2
class2 = train_x[mask, :]

# Class list for combined classes
class_list = [class1, class2]

# Get class means, covariance matrices and probabilities
mean_list, covmat_list, prob_list = get_parameters_QDA(class_list, train_x)

# Get y predictions (class that maximises the discriminant function for each observation)
ypred = QDA_class_scoring(test_x, test_y, class_list, covmat_list, mean_list, prob_list)

# Get performance report on the classification metrics
performance_report(test_y, ypred, len(class_list))


Confusion matrix for prediction:
 [[47.  1.]
 [ 2. 15.]]


Accuracy for prediction:
 0.9538461538461539


Metrics for classes
_______________________________________________________________________________
Class	|	Precision	|	Recall		|	F1 Score
_______________________________________________________________________________

Class 1 |	 0.979 		|	 0.959 		|	 0.969

Class 2 |	 0.882 		|	 0.938 		|	 0.909


Weighted F1 score:
 0.954

Macro F1 score:
 0.939
